In [4]:
# disable future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# safe imports
import os, sys       # arcgis comes with these
import datetime      # arcgis comes with this
import numpy as np   # arcgis comes with this
import pandas as pd  # arcgis comes with this

# risk imports (non-native to arcgis)
try:
    import xarray as xr  # not in arcgis
except:
    arcpy.AddError('Python library Xarray is not installed.')
    raise

# import tools
try:
    # shared folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\shared')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
    import arc, satfetcher, tools  

    # module folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\modules')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
    import vegfrax, cog  

except:
    arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
    raise

In [14]:
# try to get current map, fail if doesnt exist
aprx = arcpy.mp.ArcGISProject('CURRENT')
aprx.listColorRamps('Spectrum By Wavelength-Full Bright')[0]

In [4]:
combined_rast_lists = [
    r"C:\Users\262272G\Desktop\test\topo_rad.tif",
    r"C:\Users\262272G\Desktop\test\topo_twi.tif",
    r"C:\Users\262272G\Desktop\test\topo_tpi.tif" 
]

ds = satfetcher.load_local_rasters(rast_path_list=combined_rast_lists, 
                                   use_dask=True, 
                                   conform_nodata_to=-999)

ds

Converting rasters to an xarray dataset.
Converted raster to xarray data array: topo_rad
Converted raster to xarray data array: topo_twi
Converted raster to xarray data array: topo_tpi
Rasters converted to dataset successfully.



<xarray.Dataset>
Dimensions:   (x: 104, y: 121)
Coordinates:
  * y         (y) float64 -2.514e+06 -2.514e+06 ... -2.517e+06 -2.517e+06
  * x         (x) float64 -1.288e+06 -1.288e+06 ... -1.285e+06 -1.285e+06
Data variables:
    topo_rad  (y, x) float32 dask.array<chunksize=(121, 104), meta=np.ndarray>
    topo_twi  (y, x) float32 dask.array<chunksize=(121, 104), meta=np.ndarray>
    topo_tpi  (y, x) float32 dask.array<chunksize=(121, 104), meta=np.ndarray>
Attributes:
    transform:      (30.0, 0.0, -1288245.0, 0.0, -30.0, -2513865.0)
    crs:            +init=epsg:3577
    res:            (30.0, 30.0)
    is_tiled:       1
    nodatavals:     -999
    scales:         (1.0,)
    offsets:        (0.0,)
    descriptions:   ('Band_1',)
    AREA_OR_POINT:  Area
    DataType:       Scientific

In [5]:
ds_phenos = xr.open_dataset(r"C:\Users\262272G\Desktop\test\phenos_num.nc")
ds_phenos

<xarray.Dataset>
Dimensions:      (variable: 1, x: 104, y: 121)
Coordinates:
  * y            (y) float64 -2.514e+06 -2.514e+06 ... -2.517e+06 -2.517e+06
  * x            (x) float64 -1.288e+06 -1.288e+06 ... -1.285e+06 -1.285e+06
    spatial_ref  int32 3577
  * variable     (variable) int32 1
Data variables: (12/18)
    pos_values   (variable, y, x) float32 0.07219 0.06007 ... 0.1218 0.1121
    pos_times    (variable, y, x) float64 46.0 46.0 60.0 60.0 ... 46.0 60.0 60.0
    mos_values   (variable, y, x) float32 0.06776 0.05674 ... 0.1122 0.1007
    vos_values   (variable, y, x) float32 0.0378 0.02947 ... 0.0636 0.05862
    vos_times    (variable, y, x) float64 167.0 167.0 167.0 ... 366.0 366.0
    bse_values   (variable, y, x) float32 0.04594 0.03851 ... 0.08178 0.07416
    ...           ...
    roi_values   (variable, y, x) float32 0.0005842 0.000404 ... 0.0004972
    rod_values   (variable, y, x) float32 0.0004312 0.0003442 ... 0.0003459
    lios_values  (variable, y, x) float32 0.6947 0.5846 0.5783 ... 1.835 1.666
    sios_values  (variable, y, x) float32 0.1394 0.1302 0.1191 ... 0.377 0.3347
    liot_values  (variable, y, x) float32 1.092 0.9173 0.9193 ... 2.031 1.853
    siot_values  (variable, y, x) float32 0.2225 0.2395 0.2081 ... 0.5685 0.5049
Attributes: (12/13)
    crs:               EPSG:3577
    grid_mapping:      spatial_ref
    transform:         [ 3.00000e+01  0.00000e+00 -1.28823e+06  0.00000e+00 -...
    nodatavals:        nan
    orig_collections:  ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
    orig_bands:        ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir'...
    ...                ...
    orig_bbox:         [119.32132693 -22.77485339 119.34873642 -22.74501035]
    orig_dtype:        int16
    orig_snap_bounds:  True
    orig_cell_align:   Top-left
    orig_resample:     Nearest
    res:               30

In [58]:
try:
    # get res and crs from input
    res = tools.get_xr_resolution(ds)
    crs = tools.get_xr_crs(ds)

    # ensure dataset has proper coords, dims
    if 'spatial_ref' not in list(ds_sdm.coords):
        ds_sdm = ds_sdm.assign_coords({'spatial_ref': crs})

    # extract what we can from existing input
    ds_sdm.attrs = ds.attrs

    # do the rest
    ds_sdm = tools.build_xr_attributes(ds_sdm, res=res, crs=crs)

except:
    arcpy.AddWarning('Could not append attributes to SDM output.')

    raise


# # # # #
# notify and increment progess bar
arcpy.SetProgressorLabel('Exporting NetCDF file...')
arcpy.SetProgressorPosition(13)

# export netcdf file
tools.export_xr_as_nc(ds=ds_sdm, filename=r"C:\Users\262272G\Desktop\test\testsdm2_clean.nc")

AttributeError: 'Dataset' object has no attribute 'crs'

In [59]:
ds_pheno = xr.open_dataset(r"C:\Users\262272G\Desktop\test\phenos_num.nc")
ds_pheno['variable']

<xarray.DataArray 'variable' (variable: 1)>
array([1])
Coordinates:
    spatial_ref  int32 3577
  * variable     (variable) int32 1

# this is our testing grounds

In [79]:
ds_sdm = xr.open_dataset(r"C:\Users\262272G\Desktop\test\test\sdm_final3.nc")
ds_sdm.attrs.update({'nodatavals': np.nan})
ds_sdm.attrs.update({'crs': 'EPSG: 3577'})

In [81]:
# add classes variable dimension if missing. arcgis expects one
if 'variable' not in list(ds_sdm.dims):
    ds_sdm = ds_sdm.expand_dims('variable')
    ds_sdm = ds_sdm.assign_coords({'variable': np.array([1])})
else:
    ds_sdm['variable'] = np.array([1])

In [75]:
try:
    # get res and crs from input
    res = tools.get_xr_resolution(ds_sdm)
    crs = tools.get_xr_crs(ds_sdm)

    # ensure dataset has proper coords, dims
    if 'spatial_ref' not in list(ds_sdm.coords):
        ds_sdm = ds_sdm.assign_coords({'spatial_ref': crs})

    # extract what we can from existing input
    #ds_sdm.attrs = ds.attrs

    # do the rest
    ds_sdm = tools.build_xr_attributes(ds_sdm, res=res, crs=crs)

except:
    arcpy.AddWarning('Could not append attributes to SDM output.')
    raise

In [76]:
ds_sdm.to_netcdf(r'C:\Users\262272G\Desktop\test\test\working.nc')
xr.open_dataset(r'C:\Users\262272G\Desktop\test\test\working.nc')

TypeError: argument of type 'numpy.int32' is not iterable

In [34]:
# load netcdf. set nodata to nan to mimic dea odc
ds_temp = satfetcher.load_local_nc(nc_path=r'C:\Users\262272G\Desktop\test\test\working.nc', 
                                  use_dask=True, 
                                  conform_nodata_to=np.nan)

Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: C:\Users\262272G\Desktop\test\test\working.nc


AttributeError: 'Dataset' object has no attribute 'crs'

In [183]:
in_replacement_value = 0
in_min_value = 0.6

In [184]:
ds_sdm_check = xr.open_dataset(r"C:\Users\262272G\Desktop\mask_check\sdm.nc")
ds_mask_check = xr.open_dataset(r"C:\Users\262272G\Desktop\mask_check\mask.nc")

In [185]:
#for var in ds_mask_check.data_vars:
    #ds_mask_check = ds_mask_check[var]

In [186]:
ds_mask_check = xr.where(ds_mask_check > in_min_value, True, False)

In [188]:
ds_mask_check = ds_mask_check.to_array().squeeze(drop=True)

In [189]:
ds_sdm_check = ds_sdm_check.where(ds_mask_check, in_replacement_value)

In [190]:
ds_sdm_check

<xarray.Dataset>
Dimensions:      (variable: 1, x: 104, y: 121)
Coordinates:
  * y            (y) float64 -2.514e+06 -2.514e+06 ... -2.517e+06 -2.517e+06
  * x            (x) float64 -1.288e+06 -1.288e+06 ... -1.285e+06 -1.285e+06
    spatial_ref  int32 3577
  * variable     (variable) int32 1
Data variables: (12/18)
    pos_values   (variable, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    pos_times    (variable, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    mos_values   (variable, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    vos_values   (variable, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    vos_times    (variable, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    bse_values   (variable, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...           ...
    roi_values   (variable, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    rod_values   (variable, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    lios_values  (variable, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    sios_values  (variable, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    liot_values  (variable, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    siot_values  (variable, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/13)
    crs:               EPSG:3577
    grid_mapping:      spatial_ref
    transform:         [ 3.00000e+01  0.00000e+00 -1.28823e+06  0.00000e+00 -...
    nodatavals:        nan
    orig_collections:  ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
    orig_bands:        ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir'...
    ...                ...
    orig_bbox:         [119.32132693 -22.77485339 119.34873642 -22.74501035]
    orig_dtype:        int16
    orig_snap_bounds:  True
    orig_cell_align:   Top-left
    orig_resample:     Nearest
    res:               30